# Imports and Setup

In [ ]:
import os
import ast
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt

# module based imports
from kagglerecipes.preprocess import VoxelData
from kagglerecipes.utils import *

# Read CSV files

In [ ]:
DATA_PATH = Path(os.getcwd()).parents[1] / 'data'
TRAIN_PATH = Path(os.getcwd()).parents[1] / 'data/sample/train/'
TEST_DATA = Path(os.getcwd()).parents[1] / 'data/sample/test/'

In [ ]:
train_df = pd.read_csv(DATA_PATH / 'train_labels.csv')
test_df = pd.read_csv(DATA_PATH / 'sample_submission.csv')

def get_path(row, path_type):
    patient_id = get_patient_id(int(row.BraTS21ID))
    return f'{path_type}/{patient_id}/'

train_df['path'] = train_df.apply(lambda row: get_path(row, TRAIN_PATH), axis=1)
test_df['path'] = test_df.apply(lambda row: get_path(row, TEST_DATA), axis=1)

train_df.head(2)

,BraTS21ID,MGMT_value,path
0,165,0,e:\Kaggle\BrainTumor\kagglerecipes\data\sample...
1,267,0,e:\Kaggle\BrainTumor\kagglerecipes\data\sample...


# Get DICOM metadata

In [ ]:
meta_cols = ['SpecificCharacterSet','ImageType','SOPClassUID',
             'SOPInstanceUID','AccessionNumber','Modality', 'SeriesDescription', 
             'PatientID', 'MRAcquisitionType', 'SliceThickness', 
             'EchoTime', 'NumberOfAverages', 'ImagingFrequency', 'ImagedNucleus', 
             'MagneticFieldStrength', 'SpacingBetweenSlices', 
             'EchoTrainLength', 'PercentSampling', 'PercentPhaseFieldOfView',
             'PixelBandwidth', 'TriggerWindow', 'ReconstructionDiameter', 'AcquisitionMatrix',
             'FlipAngle', 'SAR', 'PatientPosition',
             'StudyInstanceUID', 'SeriesInstanceUID', 'SeriesNumber', 'InstanceNumber',
             'ImagePositionPatient', 'ImageOrientationPatient', 'Laterality',
             'PositionReferenceIndicator', 'SliceLocation', 'InStackPositionNumber',
             'SamplesPerPixel', 'PhotometricInterpretation', 'Rows', 'Columns', 'PixelSpacing',
             'BitsAllocated', 'BitsStored', 'HighBit', 'PixelRepresentation', 'WindowCenter',
             'WindowWidth', 'RescaleIntercept', 'RescaleSlope', 'RescaleType']

In [ ]:
# Get DICOM metadata
train_meta_df = get_all_dicom_metadata(train_df, meta_cols)
test_meta_df = get_all_dicom_metadata(test_df, meta_cols)

# Get orienation metadata
train_meta_df['Orientation'] = train_meta_df.apply(get_image_plane, axis=1)
test_meta_df['Orientation'] = test_meta_df.apply(get_image_plane, axis=1)

# Do voxel manipulation for one id

In [ ]:
folder = os.listdir(TRAIN_PATH)[0]

connect_voxel = VoxelData(os.path.join(TRAIN_PATH, folder, 'T1w'))
flair = connect_voxel.get_voxel_data(os.path.join(TRAIN_PATH, folder, 'FLAIR'))
t1wce = connect_voxel.get_voxel_data(os.path.join(TRAIN_PATH, folder, 'T1wCE'))
t2 = connect_voxel.get_voxel_data(os.path.join(TRAIN_PATH, folder, 'T2w'))